In [5]:
!pip install -q tf-models-official==2.3.0 

In [6]:
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("Alexa3Reviews.csv")

In [9]:
df.drop(inplace=True, axis=1, columns=["Unnamed: 0"])

In [10]:
df.head()

,title,text,rating,date
0,Excelente custo benefício,Sobre o produto: visual muito bonito e bem aca...,5,9 de outubro de 2019
1,Péssimo início - defeito ao tirar da caixa,O dispositivo possui 4 microfones direcionais ...,1,8 de outubro de 2019
2,Excelente produto!,Comprei o Alexa Echo Dot (terceira geração) as...,5,8 de outubro de 2019
3,Muito Satisfeito,Ainda estou me adaptando com o Echo Dot.Fiz a ...,5,13 de novembro de 2019
4,Decepção é o meu sentimento.,Acho que a propaganda desse produto está sendo...,1,23 de outubro de 2019


In [11]:
newRatings = []

for rating in df["rating"]:
    if rating==1 or rating==2 or rating==3:
        newRatings.append(0)
    else:
        newRatings.append(1)
        
df["rating"] = newRatings

In [12]:
df.head()

,title,text,rating,date
0,Excelente custo benefício,Sobre o produto: visual muito bonito e bem aca...,1,9 de outubro de 2019
1,Péssimo início - defeito ao tirar da caixa,O dispositivo possui 4 microfones direcionais ...,0,8 de outubro de 2019
2,Excelente produto!,Comprei o Alexa Echo Dot (terceira geração) as...,1,8 de outubro de 2019
3,Muito Satisfeito,Ainda estou me adaptando com o Echo Dot.Fiz a ...,1,13 de novembro de 2019
4,Decepção é o meu sentimento.,Acho que a propaganda desse produto está sendo...,0,23 de outubro de 2019


## Divisao da base em Treino, Teste e Validacao

## Transformers

In [13]:
pip install torch torchvision torchaudio

In [14]:
!pip install -q transformers

     |████████████████████████████████| 3.8 MB 8.2 MB/s 
     |████████████████████████████████| 67 kB 3.5 MB/s 
     |████████████████████████████████| 895 kB 31.7 MB/s 
     |████████████████████████████████| 596 kB 55.1 MB/s 
     |████████████████████████████████| 6.6 MB 54.7 MB/s 


In [15]:
import torch
from transformers import BertTokenizer
from transformers import AutoModel, AutoTokenizer
from transformers import BertModel, BertTokenizer

In [16]:
torch.__version__

'1.10.0+cu111'

In [17]:
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')
model = AutoModel.from_pretrained('neuralmind/bert-base-portuguese-cased')

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
def convertTextToFeature(review):
  return tokenizer.encode_plus(review,
                add_special_tokens = True, # adicao [CLS] e [SEP]
                max_length = 100, 
                pad_to_max_length = True, # padding
                return_attention_mask = True, 
              )

In [19]:
def mapTextToDict(input_ids, attention_masks, token_type_ids, label):
  return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

In [20]:
def encodeExamples(df, limit=-1):
  
  input_ids_list, token_type_ids_list, attention_mask_list, label_list = [], [], [], []

  if (limit > 0):
      df = df.take(limit)
        
  for review, label in tfds.as_numpy(df):
    
    bert_input = convertTextToFeature(review)
    
    input_ids_list.append(bert_input['input_ids'],**inputs)
    token_type_ids_list.append(bert_input['token_type_ids'])
    attention_mask_list.append(bert_input['attention_mask'])
    
    label_list.append([label])
  
  return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list, label_list)).map(mapTextToDict)

## Dividindo em Treino e teste

In [21]:
df_train = df[["text", "rating"]][:(int(0.7*len(df)))]

In [22]:
df_train

,text,rating
0,Sobre o produto: visual muito bonito e bem aca...,1
1,O dispositivo possui 4 microfones direcionais ...,0
2,Comprei o Alexa Echo Dot (terceira geração) as...,1
3,Ainda estou me adaptando com o Echo Dot.Fiz a ...,1
4,Acho que a propaganda desse produto está sendo...,0
...,...,...
3481,"O produto cumpre o prometido, a Alexa é muito ...",1
3482,Surpreendeu positivamente. Apesar de já ter vi...,1
3483,"Não tenho do que reclama, porém tem coisas que...",1
3484,Alexa veio para ajudar e facilitar minha vida....,1


In [23]:
df_test = df[["text", "rating"]][(int(0.7*len(df))):]

In [24]:
df_test

,text,rating
3486,"Eu ia comprar a 4a geração, mas como eu já con...",1
3487,Essa foi para presentear alguém que gostou mui...,1
3488,Virou a secretária do lar. As crianças e minha...,1
3489,Qualidade do som muito boa e entende muito bem...,1
3490,"Atende muitas necessidades, mas tem muito a me...",0
...,...,...
4975,Quería muito uma caixa de som para ouvir minha...,1
4976,"Muito bom, melhor ainda do que eu pense, apena...",1
4977,"O que mais gostei, é a qualidade de áudio, vol...",1
4978,"Muito boa, reconhece muito bem a voz , o volum...",1


## Preparacao dos Dados

In [25]:
def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(imdb, tokenizer):
  num_examples = len(imdb['text'])
  
  review = tf.ragged.constant([
      encode_sentence(s[:500])
      for s in imdb["text"]])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])] * review.shape[0]
  input_word_ids = tf.concat([cls, review], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor()

  type_cls = tf.zeros_like(cls)
  type_s1 = tf.ones_like(review)
  
  input_type_ids = tf.concat(
      [type_cls, type_s1], axis=-1).to_tensor()

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

def convert_to_dict(data):
  reviews = []
  labels = []
  
  for r, l in data:
    reviews.append(r.numpy())
    labels.append(l.numpy())
  
  return {'reviews': reviews, 'labels':labels}


In [27]:
tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])

[101, 102]

In [29]:
len(df_train["text"])

3486

In [30]:
dict(df_test[["text","rating"]])

{'rating': 3486    1
 3487    1
 3488    1
 3489    1
 3490    0
        ..
 4975    1
 4976    1
 4977    1
 4978    1
 4979    1
 Name: rating, Length: 1494, dtype: int64,
 'text': 3486    Eu ia comprar a 4a geração, mas como eu já con...
 3487    Essa foi para presentear alguém que gostou mui...
 3488    Virou a secretária do lar. As crianças e minha...
 3489    Qualidade do som muito boa e entende muito bem...
 3490    Atende muitas necessidades, mas tem muito a me...
                               ...                        
 4975    Quería muito uma caixa de som para ouvir minha...
 4976    Muito bom, melhor ainda do que eu pense, apena...
 4977    O que mais gostei, é a qualidade de áudio, vol...
 4978    Muito boa, reconhece muito bem a voz , o volum...
 4979    Ótimo produto, desperta, acende a lâmpada, toc...
 Name: text, Length: 1494, dtype: object}

In [31]:
df_train_enc = bert_encode(dict(df_train[["text","rating"]]), tokenizer)
df_train_labels = np.array(df_train['rating'])

df_test_enc = bert_encode(dict(df_test[["text","rating"]]), tokenizer)
df_test_labels  = np.array(df_test['rating'])

In [32]:
df_train_enc

{'input_mask': <tf.Tensor: shape=(3486, 278), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(3486, 278), dtype=int32, numpy=
 array([[0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        ...,
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(3486, 278), dtype=int32, numpy=
 array([[  101,  8053,   146, ...,     0,     0,     0],
        [  101,   231,  8985, ...,     0,     0,     0],
        [  101,  2174,  8393, ...,     0,     0,     0],
        ...,
        [  101,  2542, 15212, ...,     0,     0,     0],
        [  101,  9107,  3429, ...,     0,     0,     0],
        [  101,   231,  3608, ...,     0, 

In [33]:
df_test_enc

{'input_mask': <tf.Tensor: shape=(1494, 169), dtype=int32, numpy=
 array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_type_ids': <tf.Tensor: shape=(1494, 169), dtype=int32, numpy=
 array([[0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        ...,
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0],
        [0, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1494, 169), dtype=int32, numpy=
 array([[  101,  3396,  8544, ...,     0,     0,     0],
        [  101,  3311,   262, ...,     0,     0,     0],
        [  101,  4266,   203, ...,     0,     0,     0],
        ...,
        [  101,   231,   179, ...,     0,     0,     0],
        [  101, 12925,  3264, ...,     0,     0,     0],
        [  101,  6679,  3608, ...,     0, 

## Tamanho do vocab

In [34]:
len(tokenizer)

29794

In [35]:
tokenizer.convert_tokens_to_ids(['[CLS]', '[SEP]'])

[101, 102]

In [36]:
reviews = tf.ragged.constant([encode_sentence(s[:500]) for s in df_train['text']])

In [37]:
print("Reviews shape:", reviews.shape.as_list())

Reviews shape: [3486, None]


In [38]:
len(df_train["text"])

3486

In [39]:
for key, value in df_train_enc.items():
  print(f'{key:15s} shape: {value.shape}')

print(f'imdb_train_labels shape: {df_train_labels.shape}')

input_word_ids  shape: (3486, 278)
input_mask      shape: (3486, 278)
input_type_ids  shape: (3486, 278)
imdb_train_labels shape: (3486,)


## Aplicacao do BERT

In [41]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/keras_bert/uncased_L-12_H-768_A-12"
tf.io.gfile.listdir(gs_folder_bert)

['bert_config.json',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_model.ckpt.index',
 'vocab.txt']

In [42]:
import json

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

config_dict

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [43]:
bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=2)

In [45]:
df_batch = {key: val[:10,:500] for key, val in df_train_enc.items()}

In [47]:
bert_classifier(
    df_batch, training=True
).numpy()

array([[-0.13893074,  0.07869311],
       [-0.1524319 ,  0.29796442],
       [-0.14250611,  0.10778341],
       [-0.22884774, -0.04635305],
       [-0.31077015,  0.03122471],
       [-0.29476428,  0.12716955],
       [-0.28102067,  0.22807173],
       [-0.28095603, -0.10274608],
       [-0.23185425,  0.06583176],
       [-0.1420323 ,  0.01502384]], dtype=float32)

In [48]:
checkpoint = tf.train.Checkpoint(model=bert_encoder)
checkpoint.restore(
    os.path.join(gs_folder_bert, 'bert_model.ckpt')).assert_consumed()

In [49]:
epochs = 1
batch_size = 6
eval_batch_size = 6

train_data_size = len(df_train_labels)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

optimizer = nlp.optimization.create_optimizer(
    2e-5, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)

In [50]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

bert_classifier.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics)

bert_classifier.fit(
      df_train_enc, df_train_labels,
      validation_data=(df_test_enc, df_test_labels),
      batch_size=6,
      epochs=epochs)

581/581 [==============================] - 12232s 21s/step - loss: 0.4325 - accuracy: 0.8468 - val_loss: 0.3129 - val_accuracy: 0.8976


In [52]:
loss, accuracy = bert_classifier.evaluate(df_test_enc,df_test_labels)
print('Testing Accuracy is {} '.format(accuracy*100))

47/47 [==============================] - 756s 16s/step - loss: 0.3129 - accuracy: 0.8976
Testing Accuracy is 89.75903391838074 


In [54]:
bert_classifier.history.params

{'epochs': 1, 'steps': 47, 'verbose': 1}

In [62]:
bert_classifier.history